In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model

# Load model
model = load_model('Models/ClassifyLocations.h5')

In [3]:
def getPredictions(videoNr, xsplit, ysplit):
    """ Gets predictions from the model
    Args:
      videoNr: Number of the video to get the predictions for 
      xsplit: Number of discrete grid splits in x direction
      ysplit: Number of discrete gird splits in y direction
      
    Returns:
      Predicted locations for all image frames in the given video
    """
    
    # Load and normalize original images
    images = np.load('ImageFrames/Images_Video' + str(videoNr) + '_10.npy') 
    images = images/images.max() 
    
    # Load and normalize dilated and thresholded images
    imagesDT = np.load('ImageFrames/Images_DilatedThresholded_Video' + str(videoNr) + '_20.npy') 
    imagesDT = imagesDT/imagesDT.max() 
    
    # Load and normalize annotations for last ten locations
    annotations = np.load('AnnotationFiles/Annotations_LastTenLocations_Video' + str(videoNr) + '_' + str(xsplit) +'_' + str(ysplit) +'_10.npy')
    annotations = annotations/annotations.max()
    
    # Get prediction from model and choose the location with highest probability
    predictions = model.predict([images, imagesDT[..., np.newaxis], annotations])
    predictedLocations = np.argmax(predictions, axis=-1)
    
    return predictedLocations

In [4]:
def videoToImages(videoNr, scale_percent):
    """ Gets an array with all image frames of a given video
    Args:
      videoNr: Number of the video to get the images for 
      scale_percent: Scale factor to reduce the image for performance reasons
      
    Returns:
      Array with all image frames
    """
    
    # Calcuate dimensions of scaled images
    width = int(1920 * scale_percent / 100)
    height = int(1088 * scale_percent / 100)
    dim = (width, height)
    
    # load Video
    video = cv2.VideoCapture('Data/filmrole' + str(videoNr) + '.avi')
    if video.isOpened() == False:
        print('Error opening the video file filmrole' + str(videoNr) + '.avi')
        
    # Iterate through all image frames saving the resized frame
    success,image = video.read()
    images = np.zeros((3000,height,width,3),dtype=np.uint8)
    count = 0
    while success:
        resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        images[count] = resized
        success,image = video.read()
        count = count+1
        
    return images[0:count]

In [5]:
def createLabeledImages(videoNr, scale_percent, predictions):
    """ Gets an array with all labeled image frames of a given video
    Args:
      videoNr: Number of the video to get the predictions for 
      scale_percent: Scale factor to reduce the image for performance reasons
      predictions: Vector with predictions for all image frames
    Returns:
      Array with labeled image frames
    """

    # Get unlabeled image array and create an empty array to save the labeled images
    imagesVideo = videoToImages(videoNr,scale_percent)
    labeledImages = []

    # Loop through the predictions for all image frames to label them
    for i in range(len(predictions)):
        prediction = predictions[i]

        # If a ball is predicted in the image frame, a circle with middle point in the center of the predicted grid section is drawn
        if prediction!=0:
            xstart = int(960/xsplit*((prediction-1)%xsplit))
            xend = int(xstart + 960/xsplit)
            ystart = int(544/ysplit*np.floor((prediction-1)/xsplit))
            yend = int(ystart + 544/ysplit)
            x = xstart+(xend-xstart)/2
            y = ystart+(yend-ystart)/2
            image = cv2.circle(imagesVideo[i].copy(), (int(x),int(y)), 25, [255,0,0], 2)
            labeledImages.append(image)

        # If no ball is predicted in the image frame, the unlabeled image frame is added
        else:
            labeledImages.append(imagesVideo[i])

    return labeledImages

In [6]:
videoNr = 6
filename = 'PredictedVideos/Video' + str(videoNr) + '_Predicted_45_25.avi'
dimension = (960,544) #dimension of video that is to be created

xsplit = 45
ysplit = 25
scale_percent = 50

# Get predictions and labeled images for a given video
predictions = getPredictions(videoNr,xsplit,ysplit)
labeledImages = createLabeledImages(videoNr, scale_percent, predictions)

# Start video writer and write all labeled image frames
out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc(*'DIVX'), 25, dimension)
for i in range(len(labeledImages)):
    out.write(labeledImages[i])
out.release()
print('Predicted Video ' + str(videoNr) + ' created')

Predicted Video 6 created
